# Stimare Q function ottimale
## 1. Quanto tempo ci mette per convergere a Q*, al variare di tau
## 2. Distanza tra Q* e Q_t 
## 3. distanza tra Q_0 e Q_0 appresa tramite i valori di tau
## 4. cercare di integrare i bound trovati

Estrarre policy da Q
Confrontare due diverse Q
    Valutare diverse metriche 
Calcolare Q*
Tentativo di implementazione di un curriculum
Plot functions

In [1]:
import numpy as np
from TMDP import TMDP
from river_swim import River

from algorithms import Q_learning
from algorithms import eps_greedy
from algorithms import SARSA

In [2]:
# Test with tau=0.9
gamma = 0.9
river = River(gamma)
tau = 0.9
xi = np.ones(river.nS)*1/river.nS
tmdp = TMDP(river, xi, tau, gamma)


In [8]:
s = tmdp.reset()
M = 1000
Q = np.zeros((tmdp.nS, tmdp.nA))
ret = 0

a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])

Q = Q_learning(tmdp, s, a, Q, M)

print(Q)

[[  36.37809258   26.87824841]
 [  29.01870936   23.22352681]
 [  45.33448947   21.21340718]
 [  21.81861539   19.45363368]
 [  19.4613051    18.90720731]
 [  18.89239207 1330.56657354]]


In [17]:
tmdp_0 = TMDP(river, xi, 0, gamma)
s = tmdp_0.reset()
print(tmdp_0.tau)
M = 10000
Q = np.zeros((tmdp_0.nS, tmdp_0.nA))
ret = 0
a = eps_greedy(s, Q, 1., tmdp_0.allowed_actions[s.item()])

Q = Q_learning(tmdp_0, s, a, Q, M)

print(Q)

0
[[50.         43.36964915]
 [45.         40.97637229]
 [40.5        36.51628325]
 [36.44997517 28.95315906]
 [32.69138118  0.        ]
 [ 0.          0.        ]]


In [25]:
taus = [1 - i*0.1 for i in range(10)]
taus.append(0)
Qs = []
xi = np.ones(river.nS)*1/river.nS
gamma = 0.9
river = River(gamma)

for tau in taus:
    tmdp = TMDP(river, xi, tau, gamma)
    s = tmdp.reset()
    M = 5000
    Q = np.zeros((tmdp.nS, tmdp.nA))
    ret = 0
    a = eps_greedy(s, Q, 1., tmdp.allowed_actions[s.item()])
    Q = Q_learning(tmdp, s, a, Q, M)
    Qs.append({"tau":tau, "Q_function":Q, "env":tmdp})

for i in range(len(taus)):
    print("Tau:", Qs[i]['tau'])
    print(Qs[i]['Q_function'])

Tau: 1.0
[[2546.24052544 1669.09467684]
 [1527.27087989 2502.3344815 ]
 [2667.14446194 2217.29047658]
 [1734.52362963 2596.75756412]
 [2520.07752127 2046.24370379]
 [2119.02840077 4533.46224113]]
Tau: 0.9
[[1489.00388018 2203.48763588]
 [2271.46539956 1565.89111228]
 [2165.01420383 1326.22237223]
 [1347.98695158 2260.63403459]
 [2090.77800475 1360.08069924]
 [1565.9631211  4479.37122187]]
Tau: 0.8
[[1190.93448882 2242.45501872]
 [1168.90483049 2204.26774846]
 [1252.75548002 2174.66043123]
 [ 858.93439475 2316.46809155]
 [1569.25438134 2383.13703792]
 [1136.97627099 4673.12249027]]
Tau: 0.7
[[1789.98058169 2956.40167969]
 [2964.22369069 1822.90691102]
 [2851.45489839 2215.72131867]
 [1671.56566136 2919.24899581]
 [1888.32789221 3352.62370107]
 [2201.75538546 5341.29207292]]
Tau: 0.6
[[1583.54700732 3205.52477098]
 [3238.16778907 1674.31726427]
 [3289.10933476 1550.39975238]
 [3146.53951144 1619.05923365]
 [1540.35257825 3607.62023275]
 [1954.48557571 5047.83489614]]
Tau: 0.5
[[2999.1629